In [86]:
import pandas as pd, numpy as np

In [183]:
file_Name = 'AP - DBS Bank - Akash.xlsx'

# Data Reading

In [184]:
def csvORexcel():
    global path
    path = "C:\\Python\\read\\"+file_Name
    try:
        if file_Name.split('.')[-1].startswith('c'):
            df = pd.read_csv(path)
            return df
        elif file_Name.split('.')[-1].startswith('x'):
            df = pd.read_excel(path)
            return df
    except FileNotFoundError:
        print("The file name {0} has not found".format(path))

fn = file_Name

In [185]:
data0 = csvORexcel()
data0.head()

,Case Id,HD Name,HD AMS ID,HD ID,HD Mobile,Createdon,Sch Cat,Scheme/Doc,Scheme/Doc GUID,Status,...,Citizen Organization,Age,Citizen District,Citizen ULB,Citizen Ward,Citizen Pincode,Citizen Block,Citizen Subdistrict,Citizen Village,Family GUID
0,EP027P0F,Macha Jayalakshmi,7430,Macha.Jayalakshmi,9133018240,2023-06-03,NaN,AP-Income Certificate,DC0000YJ,Scheme/Document received,...,AP - DBS Bank Foundation,57,Anakapalli,NaN,NaN,531011.0,NaN,Atchutapuram,Pudimadaka,PG026BRH
1,EP027P1M,Macha Jayalakshmi,7430,Macha.Jayalakshmi,9133018240,2023-06-03,NaN,AP-Caste Certificate,DC0000Y4,Scheme/Document received,...,AP - DBS Bank Foundation,47,Anakapalli,NaN,NaN,NaN,NaN,Atchutapuram,Pudimadaka,PG026BSF
2,EP028W3G,Macha Jayalakshmi,7430,Macha.Jayalakshmi,9133018240,2023-06-14,NaN,AP-Caste Certificate,DC0000Y4,Scheme/Document received,...,AP - DBS Bank Foundation,46,Anakapalli,NaN,NaN,531011.0,NaN,NaN,NaN,PG027DDT
3,EP02DQA4,POLAMARASETTI SYAMALA,3013,POLAMARASETTI.SYAMALA1,9542238743,2023-07-13,NaN,AP-Caste Certificate,DC0000Y4,Scheme/Document received,...,AP - DBS Bank Foundation,18,Anakapalli,NaN,NaN,531002.0,MUNAGAPAKA,Munagapaka,T.Sirasapalle,PG02B6RJ
4,EP02ET3H,Macha Jayalakshmi,7430,Macha.Jayalakshmi,9133018240,2023-07-21,NaN,AP-Birth Certificate,DC0000Y6,Scheme/Document received,...,AP - DBS Bank Foundation,21,Anakapalli,NaN,NaN,531011.0,NaN,NaN,NaN,PG025CO3


In [186]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3629 entries, 0 to 3628
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Case Id                3629 non-null   object        
 1   HD Name                3629 non-null   object        
 2   HD AMS ID              3629 non-null   int64         
 3   HD ID                  3629 non-null   object        
 4   HD Mobile              3629 non-null   int64         
 5   Createdon              3629 non-null   datetime64[ns]
 6   Sch Cat                0 non-null      float64       
 7   Scheme/Doc             3629 non-null   object        
 8   Scheme/Doc GUID        3629 non-null   object        
 9   Status                 3629 non-null   object        
 10  Modified On            3629 non-null   datetime64[ns]
 11  Docket Submitted Date  3629 non-null   datetime64[ns]
 12  Benefit received Date  3629 non-null   datetime64[ns]
 13  HD 

In [187]:
states = {'AP':'Andhra Pradesh',
'AR':'Arunachal Pradesh',
'AS':'Assam',
'BR':'Bihar',
'BH':'Bihar',
'CT':'Chhattisgarh',
'CG':'Chhattisgarh',
'DL':'Delhi', 
'GA':'Goa',
'GJ':'Gujarat',
'HR':'Haryana',
'HP':'Himachal Pradesh',
'JH':'Jharkhand',
'KA':'Karnataka',
'KL':'Kerala',
'MP':'Madhya Pradesh',
'MH':'Maharashtra',
'MN':'Manipur',
'ML':'Meghalaya',
'MZ':'Mizoram',
'NL':'Nagaland',
'OR':'Odisha',
'PB':'Punjab',
'RJ':'Rajasthan',
'SK':'Sikkim',
'TN':'Tamil Nadu',
'TG':'Telangana',
'TR':'Tripura',
'UP':'Uttar Pradesh',
'UT':'Uttarakhand',
'WB':'West Bengal'}

# Data Cleaning

In [188]:
# Remove last row.
data0.drop(index = data0[data0.Createdon.isna()].index, inplace=True)

# Replace null values
data0['Scheme/Doc'].fillna('a', inplace=True)
data0['Citizen Name'].fillna('a', inplace=True)
data0['HD Name'].fillna('blank', inplace=True)
data0.Mobile.fillna(0, inplace=True)

# Changing status values and keeping only "Open/Submit/BR"
data0['Status'] = data0['Status'].apply(lambda x: 'Open' if x == 'Data complete' else 'Submitted' if (x=='Docket submitted' or x=='Document ready') else "Benefit Received" if x=='Scheme/Document received' else x)

# Changing Case Organization values from state initials to full state name.
data0['Case Organization'] = data0['Case Organization'].apply(lambda x: states[x[:2]])

# Renaming column "Case Organiisation" & "Case District" to "State" & "Disctrict"
data0.rename(columns={"Case Organization":"State","Case District":"District"}, inplace=True)

# Convert Mobile column from float to string for concatenation.
data0['Mobile'] = data0['Mobile'].apply(lambda x: str(x).strip())
data0['HD Mobile'] = data0['HD Mobile'].apply(lambda x: str(x).strip())
#data0['Mobile'] = data0['Mobile'].astype('int64')
data0['Mobile'] = data0['Mobile'].astype('str')
data0['HD Mobile'] = data0['HD Mobile'].astype('str')

# Checking duplicate records based on citizrn mobile number.
duplicates = data0[data0.duplicated(['Mobile'], keep='first')] # Keeping duplicate records
data0 = data0.drop(index = data0[data0.duplicated(['Mobile'], keep='first')].index) # Removing duplicate records.

# Adding column to check if HD mobile = Citizen mobile.
data0['mob_similarity'] = [True if i==j else False for i,j in zip(data0.Mobile, data0['HD Mobile'])]
mob_sim = data0[data0['mob_similarity'] == True]
data0 = data0[data0['mob_similarity'] == False]
    
# Change gender from initial letter to full form.
data0['Gender'] = data0['Gender'].apply(lambda x: 'Male' if x=='M' else 'Female' if x=='F' else 'Other' if x=='O' else x)

# Convert "Createdon", "Docket Submitted Date", "Benefit received Date" column data type to Datetime format
dt_col = ['Createdon', 'Docket Submitted Date', 'Benefit received Date', 'DOB']

for col in dt_col:
    try:
        data0[col] = pd.to_datetime(data0[col], format='mixed', errors='ignore')
    except KeyError:
        continue

# Deleting records with status "Case Aborted" and "Application rejected"
rejectedDF = data0[(data0.Status == 'Case Aborted') | (data0.Status == 'Application rejected')] # Storing prev step deleted data
data0 = data0[(data0['Status'] != 'Case Aborted') & (data0['Status'] != 'Application rejected')]

# Fill missing HD IDs with 'Not Mapped'.
data0[['HD ID', 'HD Name']] = data0[['HD ID', 'HD Name']].fillna('Not Mapped')

# Removing DFL cases data.
data0["Scheme Category"] = data0['Scheme/Doc GUID'].apply(lambda x: "DFL" if (x=="SH0009SW" or x=="SH000AG6" or x=="SH000A32" or x=="SH0009SW" or x=="SH000AG6" or x=="SH000BM6") else "E-Gov")
dfl = data0[data0["Scheme Category"] == 'DFL']
data0 = data0[data0["Scheme Category"] == 'E-Gov']


data0.reset_index(inplace=True, drop=True)
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2082 entries, 0 to 2081
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Case Id                2082 non-null   object        
 1   HD Name                2082 non-null   object        
 2   HD AMS ID              2082 non-null   int64         
 3   HD ID                  2082 non-null   object        
 4   HD Mobile              2082 non-null   object        
 5   Createdon              2082 non-null   datetime64[ns]
 6   Sch Cat                0 non-null      float64       
 7   Scheme/Doc             2082 non-null   object        
 8   Scheme/Doc GUID        2082 non-null   object        
 9   Status                 2082 non-null   object        
 10  Modified On            2082 non-null   datetime64[ns]
 11  Docket Submitted Date  2082 non-null   datetime64[ns]
 12  Benefit received Date  2082 non-null   datetime64[ns]
 13  HD 

# Sampling 20% data

In [189]:
samp = pd.DataFrame()

for hd in data0['HD ID'].value_counts().index:
    for sid in data0[data0['HD ID'] == hd]['Scheme/Doc GUID'].value_counts().index:
        conditional_data = data0[(data0['HD ID'] == hd) & (data0['Scheme/Doc GUID'] == sid)]
        if len(conditional_data) == 1:
            if len(samp) == 0:
                samp = conditional_data
            else:
                samp = pd.concat([samp, conditional_data])
        elif len(conditional_data) == 2:
            if len(samp) == 0:
                samp = conditional_data.sample(frac=0.5, random_state=1, replace=True)
            else:
                samp = pd.concat([samp,conditional_data.sample(frac=0.5, random_state=1, replace=True)], ignore_index=True)
        elif len(conditional_data) <= 5:
            if len(samp) == 0:
                samp = conditional_data.sample(frac=0.4, random_state=1, replace=True)
            else:
                samp = pd.concat([samp,conditional_data.sample(frac=0.4, random_state=1, replace=True)], ignore_index=True)
        else:
            if len(samp) == 0:
                samp = conditional_data.sample(frac=0.2, random_state=1, replace=True)
            else:
                samp = pd.concat([samp,conditional_data.sample(frac=0.2, random_state=1, replace=True)], ignore_index=True)

samp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 447 entries, 0 to 1934
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Case Id                447 non-null    object        
 1   HD Name                447 non-null    object        
 2   HD AMS ID              447 non-null    int64         
 3   HD ID                  447 non-null    object        
 4   HD Mobile              447 non-null    object        
 5   Createdon              447 non-null    datetime64[ns]
 6   Sch Cat                0 non-null      float64       
 7   Scheme/Doc             447 non-null    object        
 8   Scheme/Doc GUID        447 non-null    object        
 9   Status                 447 non-null    object        
 10  Modified On            447 non-null    datetime64[ns]
 11  Docket Submitted Date  447 non-null    datetime64[ns]
 12  Benefit received Date  447 non-null    datetime64[ns]
 13  HD Suspec

# Data Exporting

In [190]:
# Exporting data of unique records to Excel file.
with pd.ExcelWriter('C:\\Python\\export\\sample_data.xlsx') as writer:
    samp.to_excel(writer, sheet_name='Sample', index=False)
    data0.to_excel(writer, sheet_name='raw data', index=False)
    duplicates.to_excel(writer, sheet_name='duplicates', index=False)
    mob_sim.to_excel(writer, sheet_name='mobile similarity', index=False)
    dfl.to_excel(writer, sheet_name='DFL data', index=False)
    rejectedDF.to_excel(writer, sheet_name='rejected cases', index=False)